In [25]:
import pandas as pd
import numpy as np

In [26]:
# Load the dataset
data = pd.read_csv("new-bank-additional-full.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [27]:
# Step 2: Preprocessing
# Replace "unknown" with NaN
columns_to_consider = ["default", "housing", "loan", "job"]
data = data.replace("unknown", np.nan)

# Before dropping rows, print the number of rows before dropping
# Before dropping rows, print the number of rows before dropping
rows_before = len(data)
print(f"Rows before removing unknown values: {rows_before}")

# Drop rows with NaN in specified columns
data = data.dropna(subset=columns_to_consider)

# After dropping rows, print the number of rows after dropping
rows_after = len(data)
print(f"Rows after removing unknown values: {rows_after}")

# Print how many rows were dropped
rows_deleted = rows_before - rows_after
print(f"Rows dropped after removing unknown values: {rows_deleted}")

Rows before removing unknown values: 41188
Rows after removing unknown values: 31651
Rows dropped after removing unknown values: 9537


In [21]:
# Drop the 'duration' column as it leaks information
data.drop(columns=["duration"], inplace=True)

In [22]:
# Handle missing values (Imputation using mode)
data.fillna(data.mode().iloc[0], inplace=True)

In [23]:
# Encode target variable
data['y'] = data['y'].map({"yes": 1, "no": 0})

In [24]:
# Save processed data
data.to_csv("processed_bank_data.csv", index=False)